<a href="https://colab.research.google.com/github/SMTorg/smt/blob/master/tutorial/SMT_DesignSpace_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="jumbotron text-left"><b>
    
This tutorial describes how to use de DesignSpace within the SMT toolbox. 
<div>
    
    July 2024 - `SMT version 2.6.3`
  
     Paul Saves, Jasper Bussemaker (DLR), and Nathalie BARTOLI (ONERA/DTIS/M2CI)

<div class="alert alert-info fade in" id="d110">
<p>Some updates</p>
<ol> -  Manipulation of mixed DOE (continuous, integer,  categorical and hierarchical variables) </ol>
</div>

<p class="alert alert-success" style="padding:1em">
To use SMT models, please follow this link : https://github.com/SMTorg/SMT/blob/master/README.md. The documentation is available here: http://smt.readthedocs.io/en/latest/
</p>

The reference paper is available 
here https://www.sciencedirect.com/science/article/pii/S0965997818309360?via%3Dihub 

or as a preprint: http://mdolab.engin.umich.edu/content/python-surrogate-modeling-framework-derivatives

For mixed integer with continuous relaxation, the reference paper is available here https://www.sciencedirect.com/science/article/pii/S0925231219315619

In [1]:
!pip install configspace==0.6.1
#!pip install adsg_core --pre
#!pip install adsg-core[nb]

#to install smt
!pip install smt

!pip install git+https://github.com/jbussemaker/adsg-core.git@dev

#to have the latest version
!pip install smt --pre

^C
Traceback (most recent call last):
  File "/stck/psaves/miniconda3/envs/newenv1/bin/pip", line 7, in <module>
    from pip._internal.cli.main import main
  File "/stck/psaves/miniconda3/envs/newenv1/lib/python3.9/site-packages/pip/_internal/__init__.py", line 3, in <module>
    from pip._internal.utils import _log
  File "/stck/psaves/miniconda3/envs/newenv1/lib/python3.9/site-packages/pip/_internal/utils/_log.py", line 8, in <module>
    import logging
  File "/stck/psaves/miniconda3/envs/newenv1/lib/python3.9/logging/__init__.py", line 26, in <module>
    import sys, os, time, io, re, traceback, warnings, weakref, collections.abc
  File "/stck/psaves/miniconda3/envs/newenv1/lib/python3.9/weakref.py", line 25, in <module>
    import sys
KeyboardInterrupt
^C
Traceback (most recent call last):
  File "/stck/psaves/miniconda3/envs/newenv1/bin/pip", line 7, in <module>
    from pip._internal.cli.main import main
  File "/stck/psaves/miniconda3/envs/newenv1/lib/python3.9/site-packages/p

<div class="alert alert-warning" >
If you use hierarchical variables and the size of your doe greater than 30 points, you may leverage the `numba` JIT compiler to speed up the computation
To do so:
    
 - install numba library
    
     `pip install numba`
    
    
 - and define the environment variable `USE_NUMBA_JIT = 1` (unset or 0 if you do not want to use numba) 
    
     - Linux: export USE_NUMBA_JIT = 1
    
     - Windows: set USE_NUMBA_JIT = 1

</div>

In [18]:
#to ignore warning messages
import warnings
warnings.filterwarnings("ignore")

import os
import unittest
import numpy as np
from sys import argv
import matplotlib.pyplot as plt
import time
%matplotlib inline

from smt.applications import EGO
from smt.applications.ego import Evaluator
from smt.utils.sm_test_case import SMTestCase
from smt.problems import Branin, Rosenbrock, HierarchicalGoldstein
from smt.sampling_methods import FullFactorial
from multiprocessing import Pool
from smt.sampling_methods import LHS
import itertools
from smt.surrogate_models import (
    KRG,
    GEKPLS,
    KPLS,
    QP,
    MixIntKernelType,
    MixHrcKernelType,
)
from smt.applications.mixed_integer import (
    MixedIntegerContext,
    MixedIntegerSamplingMethod,
    MixedIntegerKrigingModel
)

from smt.utils.design_space import (
    DesignSpace,
    FloatVariable,
    IntegerVariable,
    OrdinalVariable,
    CategoricalVariable,
)
from adsg_core.graph.graph_edges import EdgeType
import plotly.io as pio
pio.renderers.default='notebook'


# Manipulate DOE with mixed, categorical & hierarchical variables

4 variables 
 - 1 categorical variable with 2 labels ['A', 'B'] # x0 categorical: A or B; order is not relevant
 - 1 ordinal variable with 3 levels ['C', 'D', 'E']),  # x1 ordinal: C, D or E; order is relevant
 - 1 integer variable [0,2]: 3 possibilities: 0, 1, 2
 - 1 continuous variable $\in [0, 1]$
 
 
 **Posssibility to have hierarchical variable: x1 exists only if x0 = 'A'**

In [2]:
#Instantiate the design space with all its design variables:

ds = DesignSpace([
    CategoricalVariable(['A', 'B']),  # x0 categorical: A or B; order is not relevant
    OrdinalVariable(['C', 'D', 'E']),  # x1 ordinal: C, D or E; order is relevant
    IntegerVariable(0, 2),  # x2 integer between 0 and 2 (inclusive): 0, 1, 2
    FloatVariable(0, 1),  # c3 continuous between 0 and 1
    ])

print("Number of design variables",len(ds.design_variables))
#You can define decreed variables (conditional activation):
ds.declare_decreed_var(decreed_var=1, meta_var=0, meta_value='A')  # Activate x1 if x0 == A

Number of design variables 4


In [3]:
## To give some examples
#It is also possible to randomly sample design vectors conforming to the constraints:
n = 5
x_sampled, is_acting_sampled = ds.sample_valid_x(5)

print('Data encoded: \n', x_sampled)
print('Data in initial space: \n', ds.decode_values(x_sampled))

Data encoded: 
 [[ 0.          0.          2.          0.70817685]
 [ 1.          0.         -0.          0.49876874]
 [ 1.          0.          1.          0.20578389]
 [ 0.          1.          2.          0.85209251]
 [ 0.          2.          2.          0.90432955]]
Data in initial space: 
 [['A', 'C', 2.0, 0.7081768479020769], ['B', 'C', -0.0, 0.49876874266237325], ['B', 'C', 1.0, 0.2057838947334759], ['A', 'D', 2.0, 0.8520925124490519], ['A', 'E', 2.0, 0.9043295470098508]]


In [4]:
#After defining everything correctly, you can then use the design space object 
#to correct design vectors and get information about which design variables are acting:
x_corr, is_acting = ds.correct_get_acting(x_sampled) 
print("Which variables are active \n", is_acting)

Which variables are active 
 [[ True  True  True  True]
 [ True False  True  True]
 [ True False  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


In [5]:
#If needed, it is possible to get the legacy design space definition format:
xlimits = ds.get_x_limits()
cont_bounds = ds.get_num_bounds()
unfolded_cont_bounds = ds.get_unfolded_num_bounds()
print("Limits of each variable \n", xlimits)
print("Continuous bounds with the encoding done (4 variables now) \n", cont_bounds)
print("Continuous bounds with the unfolded encoding done (5 variables now)\n", unfolded_cont_bounds)

Limits of each variable 
 [['A', 'B'], ['0', '1', '2'], (0, 2), (0, 1)]
Continuous bounds with the encoding done (4 variables now) 
 [[0 1]
 [0 2]
 [0 2]
 [0 1]]
Continuous bounds with the unfolded encoding done (5 variables now)
 [[0. 1.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 1.]]


# Manipulate DOE with continuous variables

In [6]:
#You can also instantiate a purely-continuous design space from bounds directly:
continuous_design_space = DesignSpace([(0, 1), (0, 2), (.5, 5.5)])
print("Number of design variables =",continuous_design_space.n_dv, ' or ', len(continuous_design_space.design_variables))

Number of design variables = 3  or  3


In [7]:
x_sampled_cont, is_acting_sampled_cont = continuous_design_space.sample_valid_x(5)

In [8]:
print('Data encoded: \n', x_sampled_cont)
print('Is_acting: \n', is_acting_sampled_cont)

Data encoded: 
 [[0.09133429 1.76908245 3.43167726]
 [0.16429262 0.4599719  1.68041217]
 [0.97966634 0.2432959  1.04133212]
 [0.15678333 1.9155301  2.95139151]
 [0.00421425 1.26803698 3.58326581]]
Is_acting: 
 [[ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]]


# Moving towards Architecture Design Space Graph (ADSG)

In [9]:
from adsg_core import BasicADSG, NamedNode, SelectionChoiceNode,DesignVariableNode

# Create the ADSG
adsg = BasicADSG()

ndv=13




# Create nodes
n = [NamedNode(f'N{i}') for i in range(ndv)]
n = [NamedNode("MLP"),NamedNode("Learning_rate"), NamedNode("Activation_function"),
     NamedNode("Optimizer"),NamedNode("Decay"),NamedNode("Power_update"), NamedNode("Average_start"),
     NamedNode("Running_Average_1"),NamedNode("Running_Average_2"),NamedNode("Numerical_Stability"),
     NamedNode("Nb_layers"), NamedNode("Layer_1"), NamedNode("Layer_2"), NamedNode("Layer_3"), # NamedNode("Dropout"),
     NamedNode("ASGD"),NamedNode("Adam"), NamedNode("20...40"),NamedNode("40"),NamedNode("45"),NamedNode("20...40"), NamedNode("40"),NamedNode("45"), NamedNode("20...40"),NamedNode("40"),NamedNode("45")]
adsg.add_node(n[1])
adsg.add_node(n[2])
#adsg.add_node(n[3])

# Add some edges
adsg.add_edges([
 # (n[0], n[1]),
 # (n[0], n[2]),
 # (n[0], n[3]),
  (n[3],n[10]),
  (n[14],n[4]),
  (n[14],n[5]),
  (n[14],n[6]),
  (n[15],n[7]),
  (n[15],n[8]),
  (n[15],n[9]),
])

choiceo=adsg.add_selection_choice("Optimizer_Choice", n[3], [n[14], n[15]])


choicenl=adsg.add_selection_choice("#layers", n[10], [n[11], n[12],n[13]])
#adsg.add_edges([ (n[12],n[11]), (n[13],n[12]) ])
a = []
for i in range(3) :
  a.append(NamedNode(str(25+5*i)))
b = a.copy()
b.append(n[17])
b.append(n[18])
choicel1=adsg.add_selection_choice("#neurons_1", n[11], b)
adsg.add_edges([ (n[12],choicel1), (n[13],choicel1) ])


a = []
for i in range(3) :
  a.append(NamedNode(str(25+5*i)))
b = a.copy()
b.append(n[20])
b.append(n[21])
choicel1=adsg.add_selection_choice("#neurons_2", n[12], b)
adsg.add_edges([ (n[13],choicel1) ])

a = []
for i in range(3) :
  a.append(NamedNode(str(25+5*i)))
b = a.copy()
b.append(n[23])
b.append(n[24])
choicel1=adsg.add_selection_choice("#neurons_3", n[13], b)

#adsg.add_edges([ (n[18],n[17]), (n[17],n[16]) ])
#adsg.add_edges([ (n[21],n[20]), (n[20],n[19]) ])
#adsg.add_edges([ (n[24],n[23]), (n[23],n[22]) ])

adsg.add_incompatibility_constraint([n[15], n[13]])
adsg.add_incompatibility_constraint([n[14], n[17]])
adsg.add_incompatibility_constraint([n[14], n[18]])
adsg.add_incompatibility_constraint([n[14], n[20]])
adsg.add_incompatibility_constraint([n[14], n[21]])
adsg.add_incompatibility_constraint([n[14], n[23]])
adsg.add_incompatibility_constraint([n[14], n[24]])
start_nodes = set()
start_nodes.add(n[3])
start_nodes.add(n[2])
start_nodes.add(n[1])
#start_nodes.add(n[0])



adsg.add_edges([
    (n[1], DesignVariableNode('x0', bounds=(0, 1))),
 (n[4], DesignVariableNode('x1', bounds=(0, 1))),
 (n[5], DesignVariableNode('x2', bounds=(0, 1))),
 (n[6], DesignVariableNode('x3', bounds=(0, 1))),
 (n[7], DesignVariableNode('x4', bounds=(0, 1))),
 (n[8], DesignVariableNode('x5', bounds=(0, 1))),
 (n[9], DesignVariableNode('x6', bounds=(0, 1))),
 #(n[11], DesignVariableNode('x7', options=("0", "1"))),

])

choiceo=adsg.add_selection_choice("Activation_Choice", n[2], [NamedNode("ReLU"), NamedNode("Sigmoid"),NamedNode("Tanh")])

adsg = adsg.set_start_nodes(start_nodes)
import matplotlib.pyplot as plt

adsg.render()



In [10]:
from adsg_core import GraphProcessor
gp = GraphProcessor(adsg)

print('Design variables:', gp.des_vars[0:5])
print(str(gp.des_vars[5:])[1:])
print('Objectives:', gp.objectives)
print('Constraints:', gp.constraints)

# Display some details about the encoders used for
# formulating the optimization problem
gp.get_statistics()



Design variables: [DV: #layers [3 opts], DV: Activation_Choice [3 opts], DV: Optimizer_Choice [2 opts], DV: #neurons_1 [5 opts], DV: #neurons_2 [5 opts]]
DV: #neurons_3 [5 opts], DV: x0 [0.00..1.00], DV: x1 [0.00..1.00], DV: x2 [0.00..1.00], DV: x3 [0.00..1.00], DV: x4 [0.00..1.00], DV: x5 [0.00..1.00], DV: x6 [0.00..1.00]]
Objectives: []
Constraints: []


,n_valid,n_declared,n_discrete,n_dim_cont,n_dim_cont_mean,n_exist,imp_ratio,imp_ratio_comb,imp_ratio_cont,inf_idx,dist_corr,encoder
type,,,,,,,,,,,,
option-decisions,207,1350,6,0,0.0,1,6.521739,6.521739,1.00,0.531986,0.0,complete
additional-dvs,207,0,0,7,4.0,207,1.750000,1.000000,1.75,1.000000,0.0,
total-design-space,207,2250,6,7,4.0,1,19.021739,10.869565,1.75,0.493305,0.0,complete
total-design-problem,207,2250,6,7,4.0,1,19.021739,10.869565,1.75,0.493305,0.0,complete


In [11]:
# Define the mixed hierarchical design space
from adsg_core.graph.graph_edges import EdgeType
from adsg_core import GraphProcessor
gp = GraphProcessor(adsg)
listvar= []
gvars= gp._all_des_var_data[0]
varnames= [ii.name for ii in gvars]
for i in gvars :
    if i._bounds is not None : 
        listvar.append(FloatVariable(lower=i._bounds[0],upper=i._bounds[1]) )
    elif type(i.node) is SelectionChoiceNode:
        a=((str(i._opts).replace("[", "").replace("]", "").replace(" ", "").replace("'", "").split(',')))
        listvar.append(CategoricalVariable(a) )
    else : 
        a=((str(i._opts).replace("[", "").replace("]", "").replace(" ", "").replace("'", "").split(',')))
        listvar.append(OrdinalVariable(a) )

design_space = DesignSpace(listvar)

active_vars = [i for i, x in enumerate(gp.dv_is_conditionally_active) if x]
nodelist = list(adsg._graph.nodes)
nodenamelist = [element.strip()[1:-1] for element in str(list(adsg._graph.nodes))[1:-1].replace("D[Sel:","[").replace("DV[","[").replace(" ","").split(",")  if element.strip().startswith('[') and element.strip().endswith(']')]
for i in range(np.sum(gp.dv_is_conditionally_active)) :
    meta_values = [metav for metav in  iter(adsg._graph.predecessors(nodelist[nodenamelist.index(gvars[active_vars[i]].name)]))] 
    meta_variable =  next(iter(adsg._graph.predecessors(meta_values[0])))
    while (str(meta_variable).split("[")[0] !='D') :
        meta_values =  [metav for metav in  iter((adsg._graph.predecessors(meta_values[0])))]
        meta_variable =  next(iter(adsg._graph.predecessors(meta_values[0])))
    namemetavar= str(meta_variable).replace("D[Sel:","").replace("DV[","").replace(" ","").replace("[", "").replace("]","")
    print([str(metaval)[1:-1] for metaval in meta_values] )
    design_space.declare_decreed_var(decreed_var =active_vars[i], meta_var =varnames.index(namemetavar), meta_value = [str(metaval)[1:-1] for metaval in meta_values])
 
edges= np.array(list(adsg._graph.edges.data()))
edgestype = [ edge["type"] for edge in edges[:,2]]
incomp_nodes=[]
for i,edge in enumerate(edges) : 
    if (edgestype[i]==EdgeType.INCOMPATIBILITY):
        incomp_nodes.append([edges[i][0],edges[i][1]])
def remove_symmetry(lst):
    unique_pairs = set()
    
    for pair in lst:
        # Sort the pair based on the _id attribute of NamedNode
        sorted_pair = tuple(sorted(pair, key=lambda node: node._id))
        unique_pairs.add(sorted_pair)
    
    # Convert set of tuples back to list of lists if needed
    return [list(pair) for pair in unique_pairs]
incomp_nodes=  remove_symmetry(incomp_nodes)

for pair in incomp_nodes:
    node1, node2 = pair
    vars1 = next(iter(adsg._graph.predecessors(node1)))
    while (str(vars1).split("[")[0] !='D') :
        vars1 =  next(iter(adsg._graph.predecessors(node1)))
    vars2 = next(iter(adsg._graph.predecessors(node2)))
    while (str(vars1).split("[")[0] !='D') :
        vars2 =  next(iter(adsg._graph.predecessors(node2)))
for pair in incomp_nodes:
    node1, node2 = pair
    vars1 = next(iter(adsg._graph.predecessors(node1)))
    while (str(vars1).split("[")[0] !='D') :
        vars1 =  next(iter(adsg._graph.predecessors(node1)))
    vars2 = next(iter(adsg._graph.predecessors(node2)))
    while (str(vars1).split("[")[0] !='D') :
        vars2 =  next(iter(adsg._graph.predecessors(node2)))
    namevar1= str(vars1).replace("D[Sel:","").replace("DV[","").replace(" ","").replace("[", "").replace("]","")
    namevar2= str(vars2).replace("D[Sel:","").replace("DV[","").replace(" ","").replace("[", "").replace("]","")
    design_space.add_value_constraint(var1=varnames.index(namevar1), value1=[str(node1)[1:-1]], var2=varnames.index(namevar2), value2=[str(node2)[1:-1]])  # Forbid more than 35 neurons with ASGD
     
           
    

['Layer_1', 'Layer_2', 'Layer_3']
['Layer_2', 'Layer_3']
['Layer_3']
['ASGD']
['ASGD']
['ASGD']
['Adam']
['Adam']
['Adam']


In [12]:
print(design_space)
print(gp.dv_is_conditionally_active)

Design space:
x0: Cat ['Layer_1', 'Layer_2', 'Layer_3']
x1: Cat ['ReLU', 'Sigmoid', 'Tanh']
x2: Cat ['ASGD', 'Adam']
x3: Cat ['25', '30', '35', '40', '45']
x4: Cat ['25', '30', '35', '40', '45']
x5: Cat ['25', '30', '35', '40', '45']
x6: Float (0, 1)
x7: Float (0, 1)
x8: Float (0, 1)
x9: Float (0, 1)
x10: Float (0, 1)
x11: Float (0, 1)
x12: Float (0, 1)
[False, False, False, True, True, True, False, True, True, True, True, True, True]


In [15]:
# Define the mixed hierarchical design space
design_space = DesignSpace ([
    FloatVariable (0, 1), #Learning rate
    CategoricalVariable (["ReLU","Sigmoid","Tanh"]), #3 possible choices for the activation function
    CategoricalVariable (["ASGD","Adam"]), #2 possible choices for the optimizer

    FloatVariable (0, 1), #ASGD Decay
    FloatVariable (0, 1), #ASGD Power update
    FloatVariable (0, 1), #ASGD Average start

    FloatVariable (0, 1), #Adam Running Average 1
    FloatVariable (0, 1), #Adam Running Average 2
    FloatVariable (0, 1), #Adam Numerical Stability

    IntegerVariable (1, 3), #for the number of hidden layers  (l=x9)
    OrdinalVariable (['25','30','35','40','45']), #number of hidden neurons layer 1
    OrdinalVariable (['25','30','35','40','45']), #number of hidden neurons layer 2
    OrdinalVariable (['25','30','35','40','45']), #number of hidden neurons layer 3
])

#ASGD vs Adam optimizer options activated or deactivated
design_space.declare_decreed_var(decreed_var =3, meta_var =2, meta_value =["ASGD"])
design_space.declare_decreed_var(decreed_var =4, meta_var =2, meta_value =["ASGD"])
design_space.declare_decreed_var(decreed_var =5, meta_var =2, meta_value =["ASGD"])
design_space.declare_decreed_var(decreed_var =6, meta_var =2, meta_value =["Adam"])
design_space.declare_decreed_var(decreed_var =7, meta_var =2, meta_value =["Adam"])
design_space.declare_decreed_var(decreed_var =8, meta_var =2, meta_value =["Adam"])

# Number of hidden layers: Activate x11 when x9 in [2, 3] and x12 when x9 == 3
design_space.add_value_constraint(var1=9, value1=3, var2=2, value2=["Adam"])  # Forbid 3  hidden layers with Adam
design_space.declare_decreed_var(decreed_var =10, meta_var =9, meta_value =[1,2,3])
design_space.declare_decreed_var(decreed_var =11, meta_var =9, meta_value =[2,3])
design_space.declare_decreed_var(decreed_var =12, meta_var =9, meta_value =3)
design_space.add_value_constraint(var1=10, value1=['40','45'], var2=2, value2=["ASGD"])  # Forbid more than 35 neurons with ASGD
design_space.add_value_constraint(var1=11, value1=['40','45'], var2=2, value2=["ASGD"])  # Forbid more than 35 neurons with ASGD
design_space.add_value_constraint(var1=12, value1=['40','45'], var2=2, value2=["ASGD"])  # Forbid more than 35 neurons with ASGD

